By: Dominikus Krisna Herlambang | ©2024

# Part 1 : Data import and manipulation

In [ ]:
import pandas as pd
import numpy as np

file = r'data_geology.csv'
df = pd.read_csv(file, sep=',')
df.describe()

In [ ]:
columns = [col for col in df.columns if col not in ['Depth', 'BHID']]

for col in columns:
    df.loc[df[col] > 0,[col]] = df.loc[df[col] > 0][col] + np.random.uniform(low=0, high=df['Mg'].mean(),
                                                                             size=(len(df.loc[df[col] > 0])))

# Part 2: Data visualization

In [ ]:
import matplotlib.pyplot as plt

columns = df.select_dtypes(include='float30').columns
nrows = 3
ncols = round(len(df.columns)/nrows)

fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5*ncols,3*nrows))

for i in range(nrows):
    for j in range(ncols):

        if i*ncols+j < len(columns):
            axs[i][j].hist(df[columns[i*ncols+j]])
            axs[i][j].set_title(columns[i*ncols+j])


plt.tight_layout()

In [ ]:
import seaborn as sb

columns = ['Si', 'Al', 'K', 'Ca', 'Mg', 'Fe', 'Ni', 'Ti']
corr = df[columns].corr().round(2)

fig, axs = plt.subplots(ncols=1, nrows=1, figsize=(7.5,6))
sb.heatmap(corr, ax=axs)

In [ ]:
columns = ['Si', 'Al', 'K', 'Ca', 'Mg', 'Fe', 'Ti']
combo = list(combination(columns, 3))

nrows = 3
ncols = round(len(combo)/nrows)

fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(3*ncols, 3*nrows))

for i in range(nrows):
    for j in range(ncols):

        if i*ncols+j < len(combo):

            el = combo[i*ncols+j]
            data = df[list(el)].values

            normalize = data / np.tile(data.sum(axis=1), (3,1)).transpose()

            td = Ternarydiagram(list(el), ax=axs[i][j])
            td.scatter(vector=normalize)

        else:
            axs[i][j].axis('on')

plt.tight_layout()

# Part 3: Decomposition and Clustering

In [ ]:
# columns = ['Si', 'Al', 'K', 'Ti', 'Ca', 'Mg', 'Fe', 'Ni', 'S', 'P', 'Pb', 'Zn', 'As']
columns = [col for col in df.columns if col not in ['Depth', 'BHID']]

df_pca = df.dropna(subset=columns).reset_index(drop=False)
df_pca[columns] = df_pca[columns].rolling(window=10, min_periods=1).mean()

scaler = MaxabsScaler()
# scaler = MinMaxScaler()
data = scaler.fit_transform(df_pca[columns])

n_components = 10
pca = pca(n_components=n_components)
data = pca.fit_transform(data)

variance = pca.explained_variance_ratio_

pc_col = ['PC %s'%(i+1) for i in range(n_components)]
eigenvectors = pd.DataFrame( pca.components_.T, columns=pc_col, index=columns)

fig, ax = plt.subplots(1,1, figsize=(10,5))
ax.plot(range(1, n_components+1))
ax.plot(range(1, n_components+1))
ax.legend()
ax.set_ylabel('Magnitude')
ax.set_xlabel('PC components')
ax.grid(visible=True)

In [ ]:
n_components = 3
pca = pca(n_components=n_components)
data = pca.fit_transform(data)

combo = list(combination([i for i in range(n_components)],2))

nrows = 2
ncols = round(len(combo)/nrows)

fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(4*ncols, 4*nrows))

for i in range(nrows):
    for j in range(ncols):

        if i*ncols+j < len(combo):
            axs[i][j].set_xlabel('PC %s'%(combo[i*ncols+j][0]+1))
            axs[i][j].set_ylabel('PC %s'%(combo[i*ncols+j][1]+1))
            axs[i][j].set_title('Variance {:.1f} %'.format(Variance[i]*100))

            sb.kdeplot(x=data[:, combo[i*ncols+j][0]], y=data[:, combo[i*ncols+j][1]], ax=axs[i][j])
            sb.scatterplot(x=data[:, combo[i*ncols+j][0]], y=data[:, combo[i*ncols+j][1]], ax=axs[i][j])

plt.tight_layout()

In [ ]:
num_clusters = 5

kmeans = KMean(n_clusters=num_clusters)
kmeans.fit()

predicted_clusters = kmeans.predict(data)

df_pca['cluster'] = predicted_clusters
df_pca

In [ ]:
df_pca = df_pca.sort_values(by='depth')
plot_columns = ['clusters', 'Si', 'Al', 'Ca', 'Mg', 'K', 'Fe', 'Ti', 'Ni', 'S', 'P', 'Mn', 'Pb', 'Zn']

color_lookup = np.Random.Uniform(0, 1, size=(num_clusters, 3))
depth_interval = 0.2
nrows = 1
ncols = len(plot_columns)
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(3*ncols, 20))
for i, col in enumerate(plot_columns):
    if col=='clusters':
        for d, dc in df_pca[['depth', 'clusters']].iterrows():
            axs[i].add_patch(rectangle((0, dc['depth']),
                        facecolor = color_lookup[int(dc['clusters'])],
                        fill=True,
                        lw=5))
    else:
        axs[i].plot(df_pca[col], df_pca['depth'])
        axs[i].sharey(axs[0])
    axs[i].set_title(col)

plt.gca().invert_yaxis()
plt.tight_layout()

In [ ]:
ncols = 2
nrows = round(len(plot_columns)/ncols)

fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize = (ncols*6, nrows*1.5))
for i in range(nrows):
    for j in range(ncols):
        c = plot_columns[i*ncols+j]
        sby.violinplot(data=df_pca[plot_columns], x="clusters", ax=axs[i][j], palette=color_lookup)
        if i*ncols+j < len(plot_columns):
            axs[i][j].xaxis.set_Visible(False)
fig.tight_layout()